In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor,export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error,accuracy_score,r2_score
#import xgboost as xgb

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')
df.T

,0,1,2,3,4,5,6,7,8,9,...,9694,9695,9696,9697,9698,9699,9700,9701,9702,9703
engine_displacement,170,130,170,220,210,190,240,150,250,150,...,220,280,190,220,180,140,180,220,230,270
num_cylinders,3.0,5.0,NaN,4.0,1.0,3.0,7.0,4.0,1.0,4.0,...,3.0,5.0,7.0,5.0,1.0,5.0,NaN,2.0,4.0,3.0
horsepower,159.0,97.0,78.0,NaN,140.0,NaN,127.0,239.0,174.0,123.0,...,176.0,154.0,69.0,110.0,131.0,164.0,154.0,138.0,177.0,140.0
vehicle_weight,3413.433759,3149.664934,3079.038997,2542.392402,3460.87099,2484.883986,3006.542287,3638.65778,2714.21931,3509.036569,...,2837.652898,3797.309878,3133.32921,2532.98763,3107.42782,2981.107371,2439.525729,2583.471318,2905.52739,2908.043477
acceleration,17.7,17.8,15.1,20.2,14.4,14.7,22.2,17.3,10.3,10.2,...,11.4,16.6,16.6,14.3,13.2,17.3,15.0,15.1,19.4,14.7
model_year,2003,2007,2018,2009,2009,2008,2012,2020,2016,2005,...,2013,2013,2008,2004,2005,2013,2004,2008,2011,2005
origin,Europe,USA,Europe,USA,Europe,Europe,USA,USA,Asia,USA,...,Europe,Europe,Europe,Europe,Asia,Europe,USA,USA,USA,Asia
fuel_type,Gasoline,Gasoline,Gasoline,Diesel,Gasoline,Gasoline,Gasoline,Diesel,Diesel,Gasoline,...,Gasoline,Gasoline,Gasoline,Diesel,Gasoline,Diesel,Gasoline,Diesel,Diesel,Diesel
drivetrain,All-wheel drive,Front-wheel drive,Front-wheel drive,All-wheel drive,All-wheel drive,All-wheel drive,Front-wheel drive,All-wheel drive,Front-wheel drive,Front-wheel drive,...,Front-wheel drive,Front-wheel drive,All-wheel drive,Front-wheel drive,Front-wheel drive,Front-wheel drive,All-wheel drive,All-wheel drive,Front-wheel drive,All-wheel drive
num_doors,0.0,0.0,0.0,2.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,...,-1.0,-1.0,1.0,0.0,-2.0,NaN,0.0,-1.0,1.0,-1.0


In [3]:
categorical_columns = list(df.dtypes[df.dtypes == object].index)
numerical_columns = list(df.dtypes[df.dtypes !=object].index)
print(numerical_columns)
for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(" ","_")

for col in numerical_columns:
    df[col] = df[col].fillna(0)

df_fulltrain,df_test=train_test_split(df,test_size=0.2,random_state=1)
df_train,df_val=train_test_split(df_fulltrain,test_size=0.25,random_state=1)
df_train= df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

y_train=df_train['fuel_efficiency_mpg'].values
y_val =df_val.fuel_efficiency_mpg.values
y_test = df_test.fuel_efficiency_mpg.values
del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']
df_train


['engine_displacement', 'num_cylinders', 'horsepower', 'vehicle_weight', 'acceleration', 'model_year', 'num_doors', 'fuel_efficiency_mpg']


,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors
0,120,5.0,169.0,2966.679505,13.9,2005,usa,gasoline,front-wheel_drive,-1.0
1,200,3.0,143.0,2950.822121,17.1,2013,asia,diesel,front-wheel_drive,-1.0
2,180,6.0,180.0,3078.221669,17.4,2007,usa,gasoline,all-wheel_drive,0.0
3,280,5.0,174.0,2797.991793,0.0,2016,usa,diesel,all-wheel_drive,0.0
4,250,4.0,133.0,2362.426930,16.3,2010,usa,diesel,front-wheel_drive,-1.0
...,...,...,...,...,...,...,...,...,...,...
5817,230,3.0,176.0,3430.993044,17.9,2022,europe,diesel,all-wheel_drive,0.0
5818,250,4.0,180.0,3067.664350,15.7,2010,asia,diesel,all-wheel_drive,-1.0
5819,230,2.0,182.0,3041.964593,16.7,2010,europe,diesel,all-wheel_drive,0.0
5820,180,7.0,147.0,2453.341430,15.2,2015,europe,gasoline,all-wheel_drive,0.0


In [4]:
dv= DictVectorizer(sparse=True)
train_dict = df_train.to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient = 'records')
X_val = dv.transform(val_dict)

test_dict = df_test.to_dict(orient = 'records')
X_test = dv.transform(test_dict)


In [ ]:
for n in range(10,210,10):
      rf_dt =RandomForestRegressor(n_estimators=n,random_state=1,n_jobs=-1,verbose=0)
      rf_dt.fit(X_train,y_train)
      y_pred = rf_dt.predict(X_val)
      rmse=root_mean_squared_error(y_val,y_pred)
      #scores.append(root_mean_squared_error(y_val,y_pred))
      print("RMSE for a Random forestRegressor with n=%d  is %.3f" %(n, rmse))

In [5]:
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)
#print(export_text(dt,feature_names =list(dv.get_feature_names_out())))
dv.get_feature_names_out()[np.argmax(dt.feature_importances_)]

'vehicle_weight'

In [6]:
n=10
rf_dt =RandomForestRegressor(n_estimators=10,random_state=1,n_jobs=-1,verbose=2)
rf_dt.fit(X_train,y_train)
y_pred = rf_dt.predict(X_val)


print("RMSE for a Random forestRegressor with %f estimator =%f " %(n , root_mean_squared_error(y_val,y_pred)))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 10building tree 2 of 10
building tree 3 of 10
building tree 4 of 10

building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
RMSE for a Random forestRegressor with 10.000000 estimator =0.459578 


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


To prepare a dataframe for a single vehicle entry  

In [ ]:
#for max_depth in [10,15,20,25]:
   #scores =[]
   for n in range(10,210,10):
      rf_dt =RandomForestRegressor(n_estimators=n,random_state=1,n_jobs=-1,verbose=0)
      rf_dt.fit(X_train,y_train)
      y_pred = rf_dt.predict(X_val)
      rmse=root_mean_squared_error(y_val,y_pred)
      scores.append(root_mean_squared_error(y_val,y_pred))
      print("RMSE for a Random forestRegressor with n=%d max_depth=%d is %.4f" %(n,max_depth, rmse))

RMSE for a Random forestRegressor with n=10 max_depth=10 is 0.4596
RMSE for a Random forestRegressor with n=20 max_depth=10 is 0.4536
RMSE for a Random forestRegressor with n=30 max_depth=10 is 0.4517
RMSE for a Random forestRegressor with n=40 max_depth=10 is 0.4487


KeyboardInterrupt: 

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

Train the model with these parameters:
n_estimators=10,
max_depth=20,
random_state=1,
n_jobs=-1 (optional)
Get the feature importance information from this model

In [ ]:
rf_dt=RandomForestRegressor(n_estimators=10,random_state=1,n_jobs=-1,verbose=0,max_depth=20)
rf_dt.fit(X_train,y_train)
dv.get_feature_names_out()[np.argmax(rf_dt.feature_importances_)]

'vehicle_weight'

In [ ]:
features =list(dv.get_feature_names_out())
dtrain =xgb.DMatrix(X_train,label=y_train,feature_names=features)
dval = xgb.DMatrix(X_val,label=y_val,feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'eval')]
xgb_params= {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(params = xgb_params,dtrain = dtrain,num_boost_round=100,evals=watchlist)
y_pred = model.predict(dval)
#print(y_pred)
#print(accuracy_score(y_val, y_pred))
print("XGB RMSE for eta =%f: is %.3f" % (xgb_params['eta'], root_mean_squared_error(y_val,y_pred)))

NameError: name 'xgb' is not defined

In [ ]:
features =list(dv.get_feature_names_out())
dtrain =xgb.DMatrix(X_train,label=y_train,feature_names=features)
dtest = xgb.DMatrix(X_test,label=y_test,feature_names=features)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_params= {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(params = xgb_params,dtrain = dtrain,num_boost_round=100,evals=watchlist)
y_pred = model.predict(dtest)
#print(y_pred)
#print(accuracy_score(y_val, y_pred))
print("XGB RMSE for eta =%f: is %.3f" % (xgb_params['eta'], root_mean_squared_error(y_test,y_pred)))

[0]	train-rmse:1.81393	eval-rmse:1.86841
[1]	train-rmse:1.31919	eval-rmse:1.36262


[2]	train-rmse:0.98120	eval-rmse:1.02002
[3]	train-rmse:0.75443	eval-rmse:0.79067
[4]	train-rmse:0.60680	eval-rmse:0.64379
[5]	train-rmse:0.51381	eval-rmse:0.55175
[6]	train-rmse:0.45470	eval-rmse:0.49628
[7]	train-rmse:0.41881	eval-rmse:0.46538
[8]	train-rmse:0.39534	eval-rmse:0.44691
[9]	train-rmse:0.38038	eval-rmse:0.43726
[10]	train-rmse:0.37115	eval-rmse:0.43209
[11]	train-rmse:0.36361	eval-rmse:0.42818
[12]	train-rmse:0.35850	eval-rmse:0.42663
[13]	train-rmse:0.35365	eval-rmse:0.42531
[14]	train-rmse:0.35025	eval-rmse:0.42374
[15]	train-rmse:0.34666	eval-rmse:0.42413
[16]	train-rmse:0.34459	eval-rmse:0.42417
[17]	train-rmse:0.34128	eval-rmse:0.42401
[18]	train-rmse:0.33822	eval-rmse:0.42417
[19]	train-rmse:0.33709	eval-rmse:0.42396
[20]	train-rmse:0.33553	eval-rmse:0.42333
[21]	train-rmse:0.33243	eval-rmse:0.42431
[22]	train-rmse:0.33031	eval-rmse:0.42467
[23]	train-rmse:0.32815	eval-rmse:0.42480
[24]	train-rmse:0.32670	eval-rmse:0.42479
[25]	train-rmse:0.32268	eval-rmse:0.42576
